In [2]:
from perceptron import QPerceptron
import numpy as np

In [6]:
neuron = QPerceptron(5)
# res = neuron.train()
len(neuron.input_states[0])

2

In [4]:
size = len(neuron.input_states[0][0])
w = np.random.uniform(0, np.pi/2, size)

neuron.run(0, w)

step: 00001
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 00010
x Qubit(QuantumRegister(5, 'qr'), 3)
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 00011
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 00100
x Qubit(QuantumRegister(5, 'qr'), 2)
x Qubit(QuantumRegister(5, 'qr'), 3)
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 00101
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 00110
x Qubit(QuantumRegister(5, 'qr'), 3)
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 00111
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 01000
x Qubit(QuantumRegister(5, 'qr'), 1)
x Qubit(QuantumRegister(5, 'qr'), 2)
x Qubit(QuantumRegister(5, 'qr'), 3)
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 01001
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 01010
x Qubit(QuantumRegister(5, 'qr'), 3)
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 01011
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 01100
x Qubit(QuantumRegister(5, 'qr'), 2)
x Qubit(QuantumRegister(5, 'qr'), 3)
x Qubit(QuantumRegister(5, 'qr'), 4)
step: 01101
x Qubit(QuantumRegister(5, 'qr

IndexError: index 25 is out of bounds for axis 0 with size 25

In [9]:


size = len(neuron.input_states[0][0])
w = np.random.uniform(0, np.pi/2, size)

cost = 0

for i in range(len(neuron.input_states)):
    y = neuron.run(i, w)
    cost += (neuron.input_states[i][1] - y) ** 2
    print(cost)
cost /= len(neuron.input_states)
neuron.cost_array.append(cost)

[0.51255235 0.52397501 1.18964512 ... 1.33160348 0.80382943 0.57453386]
0
[0.51255235 0.52397501 1.18964512 ... 1.33160348 0.80382943 0.57453386]


KeyboardInterrupt: 